In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import os

In [ ]:
def prepare_json(features: dict):
    feature_spec = {}
    for key, value in features.items():
        if isinstance(value, str):
            feature_spec[key] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(value, "utf-8")]))
        else:
            feature_spec[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

In [9]:
def make_predictions(features: dict):
    json_data = prepare_json(features)
    
    endpoint = "https://personality-prediction-954b9eab1c9e.herokuapp.com/v1/models/personality-model:predict"
    response = requests.post(endpoint, data=json_data)
    response.raise_for_status()

    prediction = response.json()["predictions"][0][0]
    return "Extrovert" if prediction > 0.5 else "Introvert"

In [10]:
# Contoh input 
sample_features = {
    "Time_spent_Alone": 4.0,
    "Stage_fear": "No",
    "Social_event_attendance": 4.0,
    "Going_outside": 6.0,
    "Drained_after_socializing": "No",
    "Friends_circle_size": 13.0,
    "Post_frequency": 5.0,
}

In [11]:
make_predictions(sample_features)

'Extrovert'